In [1]:

import io, os
from datetime import datetime, timezone, timedelta
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
from IPython.display import FileLink, display
import qrcode

def _fit_font(c, text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    size = max_font
    while size > min_font and pdfmetrics.stringWidth(t, font_name, size) > box_w:
        size -= 0.5
    return max(min_font, size)

def _qr_reader(value, box_size=6, border=2):
    q = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=box_size, border=border)
    q.add_data(value); q.make(fit=True)
    return ImageReader(q.make_image(fill_color="black", back_color="white").convert("RGB"))

def _neat_num(v):
    if v in (None, "", "None"): return ""
    try:
        f = float(str(v))
        return f"{f:.2f}".rstrip("0").rstrip(".")
    except:
        return str(v)

def _ph_now_str(fmt="%Y-%m-%d %H:%M"):
    try:
        ph = datetime.now(timezone(timedelta(hours=8)))
    except:
        ph = datetime.utcnow()
    return ph.strftime(fmt)

def _right_text(c, text, font, size, x_right, y, pad=0):
    t = "" if text is None else str(text)
    w = pdfmetrics.stringWidth(t, font, size)
    c.setFont(font, size)
    c.drawString(x_right - pad - w, y, t)

def render_addstock_prefilled(prefill, filename="addstock_form_a4.pdf"):
    title = "Addstock Form"
    page_size = A4
    w, h = page_size
    m = 10*mm
    pad = 2*mm
    gap = 4*mm
    left, right = m, w - m
    usable_w = right - left
    c = canvas.Canvas(filename, pagesize=page_size)
    c.setTitle(title)

    def draw_title():
        fs = _fit_font(c, title, "Helvetica-Bold", 18, 11, w - 2*m)
        c.setFont("Helvetica-Bold", fs)
        top_y = h - m - 4*mm
        c.drawCentredString(w/2, top_y, title)
        return top_y, fs

    def draw_qr_inline(cursor_y, title_fs):
        code = str(prefill.get("current_form_qr_code") or prefill.get("id") or "")
        if not code: return cursor_y - 4
        qr = _qr_reader(code, box_size=5, border=2)
        box = 16*mm
        y_top = cursor_y - title_fs - 2*mm
        x = (w - box) / 2
        c.drawImage(qr, x, y_top - box, width=box, height=box, preserveAspectRatio=True, anchor='sw', mask='auto')
        c.setFont("Helvetica-Bold", 9.5)
        c.drawCentredString(w/2, y_top - box - 10, code)
        return y_top - box - 14

    def draw_row(cursor_y, cols, row_h):
        c.setLineWidth(1.0)
        c.rect(left, cursor_y - row_h, usable_w, row_h, stroke=1, fill=0)
        x = left
        acc = 0.0
        for i, (field_key, label, ratio) in enumerate(cols):
            acc += ratio
            if i < len(cols) - 1:
                x_next = left + usable_w * acc
                c.line(x_next, cursor_y - row_h, x_next, cursor_y)
            col_w = usable_w * ratio
            lab = label
            fs = _fit_font(c, lab, "Helvetica", 9.5, 8, col_w - 2*pad)
            c.setFont("Helvetica", fs)
            c.drawString(x + pad, cursor_y - pad - fs*0.2, lab)
            val = prefill.get(field_key)
            val = "" if val is None else str(val)
            if val:
                vfs = _fit_font(c, val, "Helvetica-Bold", 12, 8, col_w - 2*pad)
                c.setFont("Helvetica-Bold", vfs)
                vy = cursor_y - (row_h/2) - (vfs*0.35)
                c.drawString(x + pad, vy, val)
            x = left + usable_w * acc
        return cursor_y - row_h

    def merge_item_quantities(items):
        allowed = ["Diesel","Regular","Premium"]
        out = {k: 0.0 for k in [a.lower() for a in allowed]}
        for it in (items or []):
            name = str(it.get("product","")).strip().lower()
            if name in out:
                try: out[name] += float(str(it.get("quantity",0)))
                except: pass
        return [{"product": "Diesel", "quantity": out["diesel"] if out["diesel"] else ""},
                {"product": "Regular","quantity": out["regular"] if out["regular"] else ""},
                {"product": "Premium","quantity": out["premium"] if out["premium"] else ""}]

    def draw_items_cards(cursor_y, items):
        cols = 3
        gap_col = 6*mm
        card_w = (usable_w - gap_col*(cols-1)) / cols
        title_h = 8*mm
        row_h = 7.0*mm
        rows = [("Product", None), ("Quantity (L)", None)]
        max_h = 0
        x = left
        for idx, it in enumerate(items):
            c.setLineWidth(1.0)
            total_h = title_h + gap + len(rows)*row_h + gap
            c.roundRect(x, cursor_y - total_h, card_w, total_h, 4, stroke=1, fill=0)
            c.setFont("Helvetica-Bold", 10)
            c.drawString(x + pad, cursor_y - title_h + 1.5, f"Item {idx+1}")
            y = cursor_y - title_h - gap
            lab_w = card_w * 0.52
            val_w = card_w - lab_w
            vals = [it.get("product",""), _neat_num(it.get("quantity"))]
            for (lab,_), val in zip(rows, vals):
                c.setFont("Helvetica", 8.7)
                c.drawString(x + pad, y - 8/2, lab)
                fs = _fit_font(c, str(val), "Helvetica-Bold", 11, 8, val_w - 2*pad)
                _right_text(c, str(val), "Helvetica-Bold", fs, x + lab_w + val_w, y - 8/2, pad)
                y -= row_h
            max_h = max(max_h, total_h)
            x += card_w + gap_col
        return cursor_y - max_h

    def draw_total(cursor_y, items):
        try:
            s = sum([float(str(i.get("quantity"))) for i in items if str(i.get("quantity")) not in ("","None")])
        except:
            s = ""
        row_h = 12*mm
        lab_w = usable_w*0.72
        val_w = usable_w - lab_w
        c.setLineWidth(1.0)
        c.rect(left, cursor_y - row_h, lab_w + val_w, row_h, stroke=1, fill=0)
        c.line(left + lab_w, cursor_y - row_h, left + lab_w, cursor_y)
        c.setFont("Helvetica-Bold", 11)
        c.drawString(left + pad, cursor_y - row_h + pad + 1, "Total Quantity (L)")
        _right_text(c, _neat_num(s), "Helvetica-Bold", 12, left + lab_w + val_w, cursor_y - row_h + pad + 1, pad)
        return cursor_y - row_h

    def draw_signatures(cursor_y):
        row_h = 18*mm
        cols = [
            ("prepared_by", "Prepared By", 0.34),
            ("verified_by", "Verified By", 0.33),
            ("approved_by", "Approved By", 0.33),
        ]
        c.setLineWidth(1.0)
        c.rect(left, cursor_y - row_h, usable_w, row_h, stroke=1, fill=0)
        acc = 0.0
        for i, (k, lab, ratio) in enumerate(cols):
            acc += ratio
            if i < len(cols)-1:
                c.line(left + usable_w*acc, cursor_y - row_h, left + usable_w*acc, cursor_y)
        x = left
        acc = 0.0
        for (k, lab, ratio) in cols:
            col_w = usable_w*ratio
            c.setFont("Helvetica", 9.5)
            c.drawString(x + pad, cursor_y - pad - 9.5*0.2, lab)
            x += col_w
        return cursor_y - row_h

    ty, tfs = draw_title()
    cursor = draw_qr_inline(ty, tfs)

    header1 = [
        ("form_name","Form Name",0.35),
        ("generated_on","Generated On",0.25),
        ("generated_by","Generated By",0.20),
        ("date","Date",0.20),
    ]
    header2 = [
        ("current_form_qr_code","Current Form QR Code",0.34),
        ("id","Form ID",0.22),
        ("location","Location",0.22),
        ("creator_employee_number","Creator Emp. No.",0.22),
    ]
    cursor = draw_row(cursor, header1, row_h=12.0*mm) - gap
    cursor = draw_row(cursor, header2, row_h=12.0*mm) - (gap + 2)

    items = merge_item_quantities(prefill.get("items"))
    cursor = draw_items_cards(cursor - 2, items) - 8
    cursor = draw_total(cursor, items) - 8
    cursor = draw_signatures(cursor)

    c.save()
    try: display(FileLink(filename))
    except: pass
    return os.path.abspath(filename)

example = {
    "form_name": "Addstock Form",
    "generated_on": _ph_now_str(),
    "generated_by": "System",
    "date": "2025-10-08",
    "current_form_qr_code": "86543219",
    "id": "AS-2025-0001",
    "location": "Loboc",
    "creator_employee_number": "12345",
    "prepared_by": "",
    "verified_by": "",
    "approved_by": "",
    "items": [
        {"product": "Diesel", "quantity": 1200},
        {"product": "Regular", "quantity": 800},
        {"product": "Premium", "quantity": 300},
        {"product": "diesel", "quantity": 50}
    ]
}

out_path = render_addstock_prefilled(example, filename="addstock_form_a4_prefilled.pdf")
display(FileLink(out_path))



h:\github4\jefstore-gasstations-backend\reportlab\addstocks\generate-filled-addstock\addstock_form_a4_prefilled.pdf

h:\github4\jefstore-gasstations-backend\reportlab\addstocks\generate-filled-addstock\addstock_form_a4_prefilled.pdf